In [1]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using Distributions
using LinearAlgebra

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 200
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int, f::Function = random_fighter)::Fighter
    if c==0
        return f()
    end
    a = random_tournament_winner(c-1, f)
    b = random_tournament_winner(c-1, f)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [5]:
function evaluate_generator2(f1, ts, np, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    t_a = random_team(f1, ts)
    f_b = pick_best_rdmly(library, t_a, np)
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            t_a = random_team(f1, ts)
            f_b = pick_best_rdmly(library, t_a, np)        
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

evaluate_generator2 (generic function with 1 method)

In [6]:
function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

compare_generator (generic function with 1 method)

## computing approx. Nash equilibria using evolutionary methods

In [7]:
# n_nash = 5
# nash_set = random_team(() -> rand_rename(random_fighter()), n_nash)

# payoffs = Matrix{Float64}(undef, (n_nash, n_nash))
# for i in 1:n_nash
#     for j in 1:n_nash
#         payoffs[i, j] = eval_battle(nash_set[i], nash_set[j])
#     end
# end

In [8]:
# n_its = 80000
# eps = 0.01/n_nash
# strategy = Vector{Float64}(undef, n_nash)
# strategy = strategy .+ 1.0/n_nash


# for i in 1:n_its
#     fitness = payoffs * strategy
#     strategy[fitness .< 0] .= strategy[fitness .< 0] .- eps
#     strategy[fitness .> 0] .= strategy[fitness .> 0] .+ eps
#     #strategy = strategy + (payoffs * strategy) .* eps
#     strategy[strategy .< 0] .= 0    
#     strategy = strategy ./ sum(strategy)
# end

In [33]:
# strategy

In [34]:
# fighters_to_df(nash_set)

In [35]:
# payoffs

In [36]:
# using fictitious play

In [61]:
size = 7000
start_size = 10
nt = 99
nash_env = Array{Fighter}(undef, size)
for i in 1:start_size
    nash_env[i] = rand(library)
end
for j in (start_size+1):size
    ff = pick_best_rdmly(library, nash_env[1:(j-1)], nt)
    nash_env[j] = ff
end

In [62]:
#fighters_to_df(env)

In [63]:
#nash_env = nash_env[1500:3000]
counter = pick_best(library, nash_env)
print(counter)
eval_battle_list(counter, nash_env)/length(nash_env)

Fighter("Olivine the Sleepy", 75, 10, 121, 17)

0.054714285714285715

## Custom methods to generate fighters

In [71]:
.75 * .52

0.39

In [72]:
.86 * .75

0.645

In [82]:
# new random generation to more closely approximate Nash
function random_scheme_a0(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    name = ""
    h = 0
    f = 0
    l = 0
    t = 0
    while budget_sofar > budget || max(h, f, l, t) > MXV 
        hf_ratio = rand() * .7 + 0.06
        l_ratio = rand()
        h_temp = rand() + 0.07
        f_temp = rand() + 0.07
        name = string("Scheme-A0 (", @sprintf("%6.3f",hf_ratio), ",", @sprintf("%6.3f",l_ratio), ",", @sprintf("%6.3f",h_temp), ",", @sprintf("%6.3f",f_temp), ")")
        if hf_ratio > 0.05
            while abs(h_temp - f_temp) < 0.3
                h_temp = rand() + 0.07
                f_temp = rand() + 0.07
            end
        end
        budget_hf = hf_ratio * budget
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 5 * l_ratio)/(h^2+f^2 + 5)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter(name, h, f, l, t)
end

#@time evaluate_generator2(random_scheme_a0, 100, 100, 50000)
@time compare_generator(random_scheme_a0, () -> rand(nash_env), 5000000)

  5.322462 seconds (79.99 M allocations: 8.870 GiB, 22.75% gc time, 1.53% compilation time)


(1.0, 0.8467312)

In [83]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    name = ""
    while budget_sofar > budget
        hf_ratio = rand()^(0.65) * .11 + 0.11
        l_ratio = rand()
        h_temp = rand() + 0.07
        f_temp = rand() + 0.07
        while abs(h_temp - f_temp) < 0.3
            h_temp = rand() + 0.07
            f_temp = rand() + 0.07
        end
        name = string("Scheme-A (", @sprintf("%6.3f",hf_ratio), ",", @sprintf("%6.3f",l_ratio), ",", @sprintf("%6.3f",h_temp), ",", @sprintf("%6.3f",f_temp), ")")
        budget_hf = hf_ratio * budget
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 2 * l_ratio)/(h^2+f^2 + 2)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter(name, h, f, l, t)
end

#@time compare_generator(random_scheme_a, random_scheme_a0, 1000000)
@time compare_generator(random_scheme_a0, () -> rand(nash_env), 5000000)

  5.176275 seconds (79.90 M allocations: 8.863 GiB, 22.93% gc time, 0.50% compilation time)


(1.0, 0.8492594)

In [28]:
@time compare_generator(random_scheme_a, random_fighter, 100000)

 42.272431 seconds (994.59 M allocations: 62.340 GiB, 13.27% gc time)


(0.9508, 1.0)

  0.049214 seconds (112.30 k allocations: 16.885 MiB, 11.61% gc time, 56.31% compilation time)


(1.0, 0.8977)

In [30]:
@time compare_generator(random_scheme_a, () -> rand(env), 10000)

  0.036271 seconds (103.36 k allocations: 16.400 MiB, 73.82% compilation time)


(1.0, 0.7631)

In [31]:
# new random generation to more closely approximate Nash
function random_scheme_b(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    opt = rand([1,1,1, 2,2, 3,3,3, 4,4, 5,5,5, 6,6, 7,7, 8,8, 9,9,9])
    flip = (rand() < 0.5)
    r1 = rand()
    r2 = rand()^(0.4)
    r3 = rand()
    name = ""
    if flip
        name = string("Scheme B-", opt, "(", @sprintf("%6.3f",r1), @sprintf("%6.3f",r2), @sprintf("%6.3f",r3), ")")
    else
        name = string("Scheme B+", opt, "(", @sprintf("%6.3f",r1), @sprintf("%6.3f",r2), @sprintf("%6.3f",r3), ")")
    end
    #           1     2     3.    4.    5.    6.    7.    8.    9. 
    h_mins =  [04.0, 05.4, 06.6, 07.5, 08.6, 09.2, 09.6, 10.3, 10.6]
    h_maxs =  [05.4, 06.6, 07.5, 08.6, 09.2, 09.6, 10.3, 10.6, 11.0]
    hl_mins = [40.0, 15.0, 12.0, 10.0, 10.0, 11.0, 15.0, 30.0, 30.0]
    hl_maxs = [65.0, 72.0, 64.0, 45.0, 48.0, 40.0, 30.0, 60.0, 60.0]
    h_raw = h_mins[opt] + (h_maxs[opt] - h_mins[opt]) * r1
    hl_raw = hl_mins[opt] + (hl_maxs[opt] - hl_mins[opt]) * r2
    l_raw = hl_raw/h_raw
    b_r = 100 - hl_raw # budget remaining raw
    f_min = b_r/15 # find given 15 tickle
    f_min = max(1, (b_r - h_raw * f_min)/15)
    f_min = max(1, (b_r - h_raw * f_min)/15)
    f_max = min((b_r - 15)/h_raw, h_raw)
    f_max = min((b_r - f_max)/h_raw, h_raw)
    f_max = min((b_r - f_max)/h_raw, h_raw)
    f_raw = f_min + (f_max - f_min) * r2
    mult = sqrt(budget/100)
    h = round(mult * h_raw)
    f = round(mult * f_raw)
    l = round(mult * l_raw)
    t = trunc((budget - h*f - h*l)/f)
    budget_sofar = h*f + h*l + f*t
    #println((h_raw, f_raw, l_raw, hl_raw, b_r, f_min, f_max))
    #println((h_raw, f_raw, l_raw, h, f, l, t, opt, hl_raw, budget_sofar, r1, r2, r3))
    if (budget - budget_sofar) >= min(h, f)
        could_add_l = (budget - budget_sofar) >= h
        could_add_t = (budget - budget_sofar) >= f
        if could_add_l && !could_add_t
            l = l+trunc((budget - budget_sofar)/h)
        end
        if !could_add_l && could_add_t
            t = t+trunc((budget - budget_sofar)/f)
        end
        if could_add_l && could_add_t
            if fpart(lraw) > fpart(traw)
                l = l+trunc((budget - budget_sofar)/h)
            else
                t = t+trunc((budget - budget_sofar)/f)
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)
    if flip
        ff = Fighter(name, f, h, t, l)
    else
        ff = Fighter(name, h, f, l, t)
    end
    return ff
end

@time compare_generator(random_scheme_b, random_scheme_a, 50000)

  0.111159 seconds (1.11 M allocations: 161.147 MiB, 28.04% gc time)


(1.0, 0.69776)

In [32]:
random_scheme_b()

Fighter("Scheme B-1( 0.380 0.551 0.706)", 36, 45, 84, 119)

## Team battles

In [50]:
nt = 1000
team_a0 = random_team(() -> random_scheme_a0(), nt)
team_b = random_team(() -> random_scheme_b(), nt)
team_a1 = random_team(() -> random_scheme_a(), nt)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), nt)
team_nash = random_team(() -> rand(nash_env), nt)
0

0

In [51]:
eval_team_battle(team_a1, team_a0)

1

In [52]:
eval_team_battle(team_a1, team_r0)

1

In [53]:
eval_team_battle(team_a0, team_r0)

1

In [57]:
eval_team_battle(team_nash, team_r0)

1

In [58]:
eval_team_battle(team_nash, team_b)

1

In [55]:
as = team_a0[1:100]
bs = team_nash[1:100]
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        println()
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Scheme-A0 ( 0.507, 0.843, 0.764, 0.982)", 45, 79, 39, 59)
         Fighter("Atalia the Pangolin", 17, 81, 11, 104)

Scheme-A0 ( 0.507, 0.843, 0.764, 0.982) lost against Atalia the Pangolin
         Fighter("Scheme-A0 ( 0.343, 0.437, 0.420, 0.531)", 68, 38, 81, 50)

Scheme-A0 ( 0.343, 0.437, 0.420, 0.531) lost against Atalia the Pangolin
         Fighter("Scheme-A0 ( 0.364, 0.614, 0.091, 0.670)", 19, 141, 21, 49)

Scheme-A0 ( 0.364, 0.614, 0.091, 0.670) lost against Atalia the Pangolin
         Fighter("Scheme-A0 ( 0.145, 0.312, 0.736, 0.854)", 25, 55, 63, 128)
Scheme-A0 ( 0.145, 0.312, 0.736, 0.854) won against Atalia the Pangolin
         Fighter("Xtreme Cricket", 15, 69, 96, 109)
Scheme-A0 ( 0.145, 0.312, 0.736, 0.854) won against Xtreme Cricket
         Fighter("Pug Abrian", 51, 89, 25, 47)
Scheme-A0 ( 0.145, 0.312, 0.736, 0.854) won against Pug Abrian
         Fighter("Olivine the Sleepy", 75, 10, 121, 17)

Scheme-A0 ( 0.145, 0.312, 0.736, 0.854) lost against Oliv

         Fighter("Scheme-A0 ( 0.590, 0.312, 0.381, 0.558)", 111, 37, 45, 24)

Scheme-A0 ( 0.590, 0.312, 0.381, 0.558) lost against Crazy Noralee
         Fighter("Scheme-A0 ( 0.502, 0.756, 0.921, 0.575)", 75, 47, 60, 42)

Scheme-A0 ( 0.502, 0.756, 0.921, 0.575) lost against Crazy Noralee
         Fighter("Scheme-A0 ( 0.279, 0.075, 1.004, 0.559)", 63, 35, 92, 57)

Scheme-A0 ( 0.279, 0.075, 1.004, 0.559) lost against Crazy Noralee
         Fighter("Scheme-A0 ( 0.748, 0.809, 0.616, 0.529)", 52, 96, 26, 38)
Scheme-A0 ( 0.748, 0.809, 0.616, 0.529) won against Crazy Noralee
         Fighter("Michai the Paper", 118, 44, 37, 10)

Scheme-A0 ( 0.748, 0.809, 0.616, 0.529) lost against Michai the Paper
         Fighter("Scheme-A0 ( 0.730, 0.818, 0.953, 0.454)", 101, 48, 39, 25)
Scheme-A0 ( 0.730, 0.818, 0.953, 0.454) won against Michai the Paper
         Fighter("Magnificent Pteranodon", 22, 36, 163, 156)
Scheme-A0 ( 0.730, 0.818, 0.953, 0.454) won against Magnificent Pteranodon
         Fighter("

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [56]:
a_i, b_i

(101, 91)